# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,masvingo,-20.0637,30.8277,23.03,25,0,5.21,ZW,1726449518
1,1,ilulissat,69.2167,-51.1000,-1.47,81,100,2.33,GL,1726449520
2,2,alice springs,-23.7000,133.8833,14.75,20,100,6.69,AU,1726449275
3,3,sao gabriel da cachoeira,-0.1303,-67.0892,26.33,72,6,1.07,BR,1726449522
4,4,albina,5.5000,-54.0500,24.50,86,80,2.03,SR,1726449523


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
city_map = city_data_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo=True,         
    tiles='OSM',
    frame_width = 800,
    frame_height = 600,
    size='Humidity',  
    color="City",     
    alpha=0.6
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# Narrow to cities that fit criteria
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 27) &  # Temp 21 to 27 degrees
    (city_data_df['Wind Speed'] < 4.5) &  # Wind less than 4.5 
    (city_data_df['Cloudiness'] == 0)     # No cloudiness
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()


# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
26,26,hamilton,39.1834,-84.5333,25.56,45,0,2.06,US,1726449548
78,78,masuguru,-11.3667,38.4167,21.78,76,0,1.89,TZ,1726449606
128,128,jamestown,42.0970,-79.2353,21.80,75,0,1.54,US,1726449434
191,191,devils lake,48.1128,-98.8651,23.30,62,0,2.57,US,1726449467
213,213,nsanje,-16.9200,35.2620,25.58,53,0,0.98,MW,1726449764


### Step 3: Create a new DataFrame called `hotel_df`.

In [20]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
# YOUR CODE HERE
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
26,hamilton,US,39.1834,-84.5333,45,
78,masuguru,TZ,-11.3667,38.4167,76,
128,jamestown,US,42.0970,-79.2353,75,
191,devils lake,US,48.1128,-98.8651,62,
213,nsanje,MW,-16.9200,35.2620,53,
238,brookings,US,44.3114,-96.7984,72,
249,vila franca do campo,PT,37.7167,-25.4333,81,
292,griffith,US,41.5284,-87.4237,75,
343,palmas,BR,-10.2128,-48.3603,39,
383,cascais,PT,38.6979,-9.4215,54,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",  # Select category
    "limit": 1,  # Closest hotel
    "apiKey": geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hamilton - nearest hotel: North Vista Manor
masuguru - nearest hotel: No hotel found
jamestown - nearest hotel: DoubleTree Jamestown
devils lake - nearest hotel: Holiday Inn Express
nsanje - nearest hotel: No hotel found
brookings - nearest hotel: Quality Inn
vila franca do campo - nearest hotel: Vinha d'Areia Beach Hotel
griffith - nearest hotel: Quality Inn & Suites Hammond
palmas - nearest hotel: Hotel Triângulo Mineiro
cascais - nearest hotel: Cascais Hotel
metlili chaamba - nearest hotel: No hotel found
al bawiti - nearest hotel: Old Oasis Hotel
pass christian - nearest hotel: Bay Town Inn B


,City,Country,Lat,Lng,Humidity,Hotel Name
26,hamilton,US,39.1834,-84.5333,45,North Vista Manor
78,masuguru,TZ,-11.3667,38.4167,76,No hotel found
128,jamestown,US,42.0970,-79.2353,75,DoubleTree Jamestown
191,devils lake,US,48.1128,-98.8651,62,Holiday Inn Express
213,nsanje,MW,-16.9200,35.2620,53,No hotel found
238,brookings,US,44.3114,-96.7984,72,Quality Inn
249,vila franca do campo,PT,37.7167,-25.4333,81,Vinha d'Areia Beach Hotel
292,griffith,US,41.5284,-87.4237,75,Quality Inn & Suites Hammond
343,palmas,BR,-10.2128,-48.3603,39,Hotel Triângulo Mineiro
383,cascais,PT,38.6979,-9.4215,54,Cascais Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
city_map_new = hotel_df.hvplot.points(
    'Lng', 'Lat',               # Longitude and Latitude for plotting
    geo=True,                   # Enables geographic plotting
    tiles='OSM',                # OpenStreetMap tiles as background
    size='Humidity',            # Size of the point based on humidity
    color='City',               # Color of the points
    alpha=0.6,                  # Transparency
    hover_cols=['City', 'Country', 'Hotel Name']  # hover info
)

# Display the map
city_map_new

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)